# 3. Import biosphere3 and ecoinvent

Now that you know how to work with the foreground system, it's time to learn how to work with the background system. In particular it is useful to import and work with two databases: _biosphere3_ that contains all the exchanges and impact assessment methods, and _ecoinvent_. 

#### Read this before starting

This tutorial shows an "old school" way of importing ecoinvent, to be used when you have the files on your hard disk.

There is however an [updated way of doing it](https://github.com/massimopizzol/advanced-lca-notebooks/blob/main/Specific-bw-tasks/Import-ecoinvent-easily-with-interface.py) that fetches automatically files etc., it works smoother. Requires credentials. Check the linked tutorial.

#### Importing ecoinvent 

You need to get the ecoinvent files first, see steps below:

1. Open the ecoinvent website and [login](https://ecoquery.ecoinvent.org/3.10/cutoff/search) with your username and password.
2. You should read somewhere: _To download LCI and LCIA cumulative matrices click here_. Click there.
3. Select _ecoinvent 3.9.1_
4. Download the file `ecoinvent 3.9.1_consequential_ecoSpold02.7z` in a folder of yours. Make sure you remember the full path to this directory. E.g. I have downloaded the file in:
_/Users/massimo/Documents/Databases/ecoinvent v3.9.1_
5. The file you have downloaded is a compressed archive of many files (like with winzip or winrar). Extract the files from the .7z archive, e.g. by double clicking it. If it does not work, install a software that can do that. E.g. for mac users you can either download [theunarchiver](https://theunarchiver.com/) or, if you are using brew, just open terminal and do `brew install p7zip` and then from terminal find the folder and do `7z x 'ecoinvent 3.9.1_consequential_ecoSpold02.7z'` (here the [p7zip instructions](https://wiki.archlinux.org/index.php/p7zip) in case).
6. Now you an run the cells below. __Make sure you change the line of code that specifies the directory__ by writing the directory on your computer where you have extracted the files. For example, I have extracted the files in a folder called "datasets". The path to this folder is: _/Users/massimo/Documents/Databases/ecoinvent v3.9.1/datasets_ You will see this same line in the script and you need to change it with your directory. 

In [4]:
# Import brightway2.5 packages
import bw2calc as bc
import bw2data as bd
import bw2io as bi

In [5]:
bd.projects # check what project you have 
# bd.projects.delete_project('advlca23', delete_dir=True) # if you want a fresh start

Brightway2 projects manager with 49 objects, including:
	ALIGNED
	B4B18
	BTC02
	BTC_Att_LCA
	Biowood
	C3BO
	CCU
	ConseqUncertainty
	GSA
	Giovanni
Use `sorted(projects)` to get full list, `projects.report()` to get
	a report on all projects.

In [6]:
bd.projects.set_current('advlca25') # Still working in the same project
bd.databases
# bd.databases.clear() # For a fresh start (Risky command! clears all your existing databases)

Databases dictionary with 2 object(s):
	testbiosphere
	testdb

Before importing ecoinvent, we need to make a default setup of Brightway2. This means importing all the environmental exchanges and all the LCIA methods. Then when we import ecoinvent the ecoinvent activities will be linked to this database of environmental exchanges, just like in the previous example wiht the product system of H&S 2002.

In [7]:
# Import the biosphere3 database
bi.bw2setup() # This will take some time

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds


100%|████████████████████████████████████| 4709/4709 [00:00<00:00, 15543.46it/s]


Vacuuming database 
Created database: biosphere3
Creating default LCIA methods

Wrote 762 LCIA methods with 227223 characterization factors
Creating core data migrations



In [4]:
# If you encounter the following error , uncomment the following code.
# Error: "ValueError: Method ('CML v4.8 2016 no LT', 'acidification no LT', 'acidification (incl. fate, average Europe total, A&B) no LT') already exists. Use ``overwrite=True`` to overwrite existing methods"

# methods_to_delete = list(bd.methods)
# for method in methods_to_delete:
#     m = bd.Method(method)
#     m.deregister()
# del bd.databases["biosphere3"]

We are going to use version 3.9.1 of ecoinvent, consequential model, for this course.

In [9]:
# Import ecoinvent

# You need to change the line below with the directory where you have saved ecoinvent
ei391dir = "/Users/massimo/Documents/Databases/ecoinvent v3.9.1/ecoinvent 3.9.1_consequential_ecoSpold02/datasets"
#ei391dir = "/Users/bp45th/Downloads/ecoinvent 3.9.1_consequential_ecoSpold02/datasets"

if 'ecoinvent 3.9.1 conseq' in bd.databases:
    print("Database has already been imported")
else:
    ei391 = bi.SingleOutputEcospold2Importer(ei391dir, 'ecoinvent 3.9.1 conseq') # You can give it another name of course
    ei391.apply_strategies()
    ei391.statistics()

#ei391.drop_unlinked(True) # Use this line if you get an "unlinked exchanges" error
ei391.write_database() # This will take some time.

Extracting XML data from 18856 datasets
Extracted 18856 datasets in 96.21 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

100%|████████████████████████████████████| 18856/18856 [00:27<00:00, 689.33it/s]


Vacuuming database 
Created database: ecoinvent 3.9.1 conseq


Brightway2 SQLiteBackend: ecoinvent 3.9.1 conseq

In [11]:
bd.databases # you should now see both "biosphere3" and "ecoinvent 3.9.1 conseq"

Databases dictionary with 4 object(s):
	biosphere3
	ecoinvent 3.9.1 conseq
	testbiosphere
	testdb

# Navigate biosphere3 and ecoinvent

A key difference compared to previous exercises is that in ecoinvent each activity and exchange is defined by a **code** which are unique identifiers. So it is important to learn how to find both activity code and name and how to match them _(Actually we used the codes also in the previous lectures but they were identical to the activity names for simplicity)_.

One key and I would say __fundamental__ aspect to remember is that __codes works across bw installations__. This means that the code (actually it's a unique identifier or [UUID](https://en.wikipedia.org/wiki/Universally_unique_identifier), I think) is the same on your computer and on another person's computer. This is extremely important to be able to __share data__.

In [12]:
# Search stuff in biosphere
bd.Database("biosphere3").search("carbon dioxide") # there is more than one activity with this name. Only code is univocal.

['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, non-fossil, resource correction' (kilogram, None, ('natural reso

In [14]:
CO2 = bd.Database("biosphere3").get("349b29d1-3e58-4c66-98b9-9d1a076efd2e") # This code works across bw2.5 installations, 
                                                                    ### i.e. is univocal for biosphere3 everywhere
print(CO2['name']) # there is more than one activity with this name. Only code is univocal.
print(CO2['code'])

Carbon dioxide, fossil
349b29d1-3e58-4c66-98b9-9d1a076efd2e


In [15]:
# Search stuff in ecoinvent

# Search by keyword
mydb = bd.Database('ecoinvent 3.9.1 conseq')
#mydb.search("*") # to search everything
mydb.search("transport freight euro5")

#bd.Database('ecoinvent 3.9.1 conseq').search("transport") # gives the same result obviously

['transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, BR, None),
 'market for transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, RER, None),
 'market for transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, BR, None),
 'market for transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, RoW, None),
 'transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, RER, None),
 'transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, RoW, None),
 'transport, freight, lorry 3.5-7.5 metric ton, EURO5' (ton kilometer, BR, None),
 'transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, BR, None),
 'transport, freight, lorry 16-32 metric ton, EURO5' (ton kilometer, BR, None),
 'transport, freight, lorry, all sizes, EURO5 to generic market for transport, freight, lorry, unspecified' (ton kilometer, BR, None),
 'transport, freight, lorry, all sizes, EURO5 to generic market for transport, freight, lorry, unspecified' (ton kilo

In [69]:
# explore activities

activity_name = 'electricity'

# Same but different:
for activity in bd.Database("ecoinvent 3.9.1 conseq").search(activity_name, limit = 5):  
    print(activity)
    print(activity['code'])
    print(activity['id'])

'electricity production, oil' (kilowatt hour, CO, None)
c75f78213b09d028d43b259f776bc86d
6912
'electricity production, oil' (kilowatt hour, US-WECC, None)
ee9f91e5c13b3ee8a73237d9864cbe18
10580
'electricity production, oil' (kilowatt hour, CN-FJ, None)
a78cb6a46466fd367d1d974d5281e8d0
5484
'electricity production, oil' (kilowatt hour, CN-GZ, None)
4ab58c28574b3f858a8a8568a39d28f5
16983
'electricity production, oil' (kilowatt hour, CN-GX, None)
746dfaa2dd5693ec139b8a558b082095
10501


#### Note: difference between 'code' and 'id' (introduced in bw25)

In bw2.5 there is a new field, the "id" field. This is also an unique identifier of each exchange, but it's an integer not a string. Specifically, the "id" is basically a coordinate, it is the row/(column number of this exchange in the technology matrix (or in the biosphere matrix if a biosphere excahange).

Careful that (but I am not sure) differently from the 'code' the 'id' might not be univocal across bw installations!

In [34]:
act = bd.Database("ecoinvent 3.9.1 conseq").get('c75f78213b09d028d43b259f776bc86d') 
print(type(act['code'])) # the code is a string
print(type(act['id'])) # the id is an integer

<class 'str'>
<class 'int'>


In [70]:
# Try this        
for activity in bd.Database("biosphere3").search('heat production'):  
    print(activity)
    print(activity['code']) # Can you explain this result?

In [71]:
# you can be much more specific in your search:
for activity in bd.Database("ecoinvent 3.9.1 conseq").search(activity_name, filter={"location" : 'DK'}, limit = 5):
    print(activity)
    print(activity['code'])
    print(activity['id'])

Excluding 3525 filtered results
'electricity production, oil' (kilowatt hour, DK, None)
2d247f0b80d8af196724430602b0d86d
19163
'market for electricity, low voltage' (kilowatt hour, DK, None)
ce40dd006e462e720eb669efbc31dd59
7794
'market for electricity, high voltage' (kilowatt hour, DK, None)
1b3abeab9ce01846ca43ab75f0fbfbf7
16179
'market for electricity, medium voltage' (kilowatt hour, DK, None)
f4dc7d2b1d70e6c0f929ec5231c085e0
11216
'electricity, high voltage, import from NO' (kilowatt hour, DK, None)
23185c39f5391a0c31290b0cf8c41e6d
20811


Now you know how to find activities. What about **selecting** activities?

In [72]:
# If you know the code (e.g. found with method above) it's simple.        
mycode = 'ce40dd006e462e720eb669efbc31dd59'
myact = bd.Database("ecoinvent 3.9.1 conseq").get(mycode)
#myact = Database("biosphere3").get(mycode)  # Not working of course...

print(myact['name'])

market for electricity, low voltage


In [73]:
myact.id # access the id directly

7794

In [74]:
myact.code # this won't work though. It's only for excahnges (see further below)

AttributeError: 'Activity' object has no attribute 'code'

In [75]:
myact._data # a lot of detail

{'comment': "This is a market activity. Each market represents the consumption mix of a product in a given geography, connecting suppliers with consumers of the same product in the same geographical area. Markets group the producers and also the imports of the product (if relevant) within the same geographical area. They also account for transport to the consumer and for the losses during that process, when relevant.\nThis dataset describes the electricity available on the high voltage level in Denmark. This is done by showing the transmission of 1kWh electricity at high voltage.\nTechnology:  Average technology used to transmit and distribute electricity. Includes underground and overhead lines, as well as air-, vacuum- and SF6-insulated high-to-medium voltage switching stations. Electricity production according to related technology datasets.\nTime period:  The 'Start of Period' and 'End of Period' do not refer to the year for which this market is valid. See general comment for the y

In [78]:
# Check the exchanges in one activity
for i in list(myact.exchanges()):  # Epxlore the activity as usual
    print(i['type'])
    print(i)
    print(i['input'])
    print('-------')

production
Exchange: 1.0 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, DK, None) to 'market for electricity, low voltage' (kilowatt hour, DK, None)>
('ecoinvent 3.9.1 conseq', 'ce40dd006e462e720eb669efbc31dd59')
-------
technosphere
Exchange: 8.74048809653223e-08 kilometer 'market for distribution network, electricity, low voltage' (kilometer, GLO, None) to 'market for electricity, low voltage' (kilowatt hour, DK, None)>
('ecoinvent 3.9.1 conseq', 'a0292b0328a3b02bc80b97b87f293152')
-------
technosphere
Exchange: 0.0229 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, DK, None) to 'market for electricity, low voltage' (kilowatt hour, DK, None)>
('ecoinvent 3.9.1 conseq', 'ce40dd006e462e720eb669efbc31dd59')
-------
technosphere
Exchange: 7.36096869893038e-05 kilowatt hour 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, DK, None) to 'market for electricity, low voltage' (kilowatt hou

In [79]:
# If you know the name of the activity and want to select it:
activity_name = 'market for electricity, low voltage'
    
for activity in bd.Database("ecoinvent 3.9.1 conseq"):  # can you find an easier way? I couldn't
    if activity['name'] == activity_name:
        myact = bd.Database("ecoinvent 3.9.1 conseq").get(activity['code'])

myact  # Careful! Might not return the danish market. Not what I wanted! 

'market for electricity, low voltage' (kilowatt hour, ZA, None)

In [81]:
# A more specific search
for activity in bd.Database("ecoinvent 3.9.1 conseq"):  
    if activity['name'] == activity_name and activity['location'] == "DK":  # need to be specific...
        myact = bd.Database("ecoinvent 3.9.1 conseq").get(activity['code'])
myact  # alright

'market for electricity, low voltage' (kilowatt hour, DK, None)

In [82]:
# Explore exchanges
myexc = list(myact.exchanges())[1]

In [83]:
# All the metadata of an exchange 
for i in myexc:
    print(i)

flow
type
name
classifications
production volume
properties
activity
unit
comment
formula
amount
pedigree
uncertainty type
loc
scale
scale without pedigree
input
output


In [85]:
# access exchange metadata
print(myexc['type'], myexc['output'], myexc['input'])

technosphere ('ecoinvent 3.9.1 conseq', 'ce40dd006e462e720eb669efbc31dd59') ('ecoinvent 3.9.1 conseq', 'a0292b0328a3b02bc80b97b87f293152')


In [86]:
# again exchange metadata
print(myexc['pedigree'])

{'reliability': 3, 'completeness': 2, 'temporal correlation': 4, 'geographical correlation': 4, 'further technological correlation': 3}


# Calculate with biosphere3 and ecoinvent 

Ok now we can run an LCA with a dataset from ecoinvent.

In [91]:
list(bd.methods)[0:5] # remove [0:5] to see the very long list of all methods.

[('simplemethod', 'imaginaryendpoint', 'imaginarymidpoint'),
 ('CML v4.8 2016 no LT',
  'acidification no LT',
  'acidification (incl. fate, average Europe total, A&B) no LT'),
 ('CML v4.8 2016 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: freshwater no LT',
  'freshwater aquatic ecotoxicity (FAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: marine no LT',
  'marine aquatic ecotoxicity (MAETP inf) no LT')]

In [92]:
# First select a method from the list (use 'list(methods)' to see all of them)
mymethod = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
mymethod

('IPCC 2013', 'climate change', 'global warming potential (GWP100)')

In [97]:
myact = bd.Database("ecoinvent 3.9.1 conseq").get('ce40dd006e462e720eb669efbc31dd59')
myact

'market for electricity, low voltage' (kilowatt hour, DK, None)

In [102]:
functional_unit = {myact : 1}
lca = bc.LCA(demand=functional_unit, method=mymethod) #run LCA calculations again with method
lca.lci()
lca.lcia()
print(lca.score) # What is the unit? Find out! (alright it's bd.methods[mymethod])

0.04779408709945726


In [103]:
bd.methods[mymethod]

{'description': '',
 'filename': 'LCIA_Implementation_3.9.xlsx',
 'unit': 'kg CO2-Eq',
 'abbreviation': 'ipcc-2013cg.c44286db43e989e224357edf263706ce',
 'num_cfs': 207,
 'geocollections': ['world']}

# Exercise (at home)
Link the emissions of your previously defined foreground system to the biosphere3 database, and link some of the ecoinvent database activities to your foreground system. Run the calculations and get a carbon footprint with the ILCD climate change method.